In [6]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession 
from math import ceil,fsum
spark = SparkSession.builder.master("local[4]").appName("session1").getOrCreate()
sc = spark.sparkContext

21/11/08 19:53:26 WARN Utils: Your hostname, pamousso-Latitude-E5530-non-vPro resolves to a loopback address: 127.0.1.1; using 192.168.1.13 instead (on interface wlp2s0)
21/11/08 19:53:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/08 19:53:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
## Part IV: Matrix Operations

In [38]:
file_V = sc.textFile("V.txt")
file_W = sc.textFile("W.txt")
file_M = sc.textFile("M.txt")

In [93]:
def cleanMat(mat):
    vect = [int(mat[i]) for i in range(len(mat)-1)]
    vect.append(float(mat[-1]))
    vect=tuple(vect)
    return vect

file_V_clean = file_V.map(lambda line: (line.split())).map(cleanMat)
file_W_clean = file_W.map(lambda line: (line.split())).map(cleanMat)
file_M_clean = file_M.map(lambda line: (line.split())).map(cleanMat)
file_W_clean.collect()
# V = [(1, 1.5), (2, 5.0), (4, 1.3), (7, 3.0)]
# W = [(1, -1.5), (2, 2.0), (3, 2.3), (4, 2.0), (6, 2.5)]

[(1, -1.5), (2, 2.0), (3, 2.3), (4, 2.0), (6, 2.5)]

In [10]:
#2) The L2 norm of a vector V is norm(V) = sqrt(V1*V1 + V2*V2 + ... +Vn*Vn)
file_V_l2_norm = round(sqrt(file_V_clean.map(lambda line: pow(line[-1],2)).sum()),3)
file_V_l2_norm

In [72]:
# 3) We want to compute the vector sum U = V + W. 
file_V__W = file_V_clean.union(file_W_clean)
file_V_W_sum = file_V__W.reduceByKey(lambda a,b : a+b).filter(lambda line: line[1]!=0.0).sortByKey()
file_V_W.collect()

6.16

In [84]:
# 4) We want to compute the dot product of two vectors
file_V_W_dot = file_V_clean.join(file_W_clean).map(lambda line : line[1])
file_V_W_dot =file_V_W_dot.map(lambda line:(line[0]*line[1])).sum()

[(4, (1.3, 2.0)), (1, (1.5, -1.5)), (2, (5.0, 2.0))]

10.35

[(1, ((1, 1.5), (1, -1.5))),
 (2, ((2, 5), (2, 2))),
 (3, ((4, 1.3), (3, 2.3))),
 (4, ((7, 3), (4, 2)))]

61.74

### Now, we want to compute the matrix vector product T = M.V.
The vector T must also be represented in a RDD using a sparse coding.

    [(1, 1.5), 
     (2, 5), 
     (4, 1.3), =>(3, 1.3)
     (7, 3)] => (4, 3)

    [(1, 1, 3.2),
     (1, 2, 2.4),
     (1, 3, 7),
     (1, 4, 2),
     (2, 2, 7.1),
     (2, 3, -1),
     (3, 3, 1)] 
     
      1   2   3  4
    1 3.2 2.4 7  2 => 19.4 =>1
    2 0  7.1 -1  0 => 35.5 =>2
    3 0   0   1  0 => 0    =>3
      ------------
      1.5 5  0  1.3
      
 [(1,1) (1.5, 3.2)), (1,2), (5.0, 2.4)), (2, (5.0, 7.1)), (1,4), (1.3, 2.0)),(2,2),(5,2))
    


In [115]:
file_M_bis =file_M_clean.map(lambda line: ((line[0],line[1]),line[2]))

[((1, 1), 3.2),
 ((1, 2), 2.4),
 ((1, 3), 7.0),
 ((1, 4), 2.0),
 ((2, 2), 7.1),
 ((2, 3), -1.0),
 ((3, 3), 1.0)]

In [156]:
file_M_bis.collect()

[((1, 1), 3.2),
 ((1, 2), 2.4),
 ((1, 3), 7.0),
 ((1, 4), 2.0),
 ((2, 2), 7.1),
 ((2, 3), -1.0),
 ((3, 3), 1.0)]

In [188]:
def dot_prod(vector):
    mat = file_M_bis.collect()
    indice=[1 for i in range(len(mat)) if((mat[i][0][0]!=mat[i-1][0][0]))]
    rddV = []
    for i in range(len(indice)):
        rddV.append(vector.map(lambda line:((i+1,line[0]),line[1])).collect())
    rddV = sc.parallelize(rddV)
    return rddV.flatMap(lambda line: line)
v = dot_prod(file_V_clean)

[((1, 1), 1.5),
 ((1, 2), 5.0),
 ((1, 4), 1.3),
 ((1, 7), 3.0),
 ((2, 1), 1.5),
 ((2, 2), 5.0),
 ((2, 4), 1.3),
 ((2, 7), 3.0),
 ((3, 1), 1.5),
 ((3, 2), 5.0),
 ((3, 4), 1.3),
 ((3, 7), 3.0)]

In [157]:
m_v_ = file_M_bis.join(v).sortByKey()
m_v=m_v_.map(lambda line: (line[0][0], line[1])).map(lambda line: (line[0],line[1][0]*line[1][1]))
m_v.reduceByKey(lambda a,b : round(a+b,3))

[(1, 19.4), (2, 35.5)]

In [158]:
m_v_.collect()

[((1, 1), (3.2, 1.5)),
 ((1, 2), (2.4, 5.0)),
 ((1, 4), (2.0, 1.3)),
 ((2, 2), (7.1, 5.0))]

In [212]:
# 7) (optional) write a program that computes the largest eigenvalue of matrix M. 
import random
v0 = sc.parallelize([(i,random.randrange(1,10000)) for i in range(1,4)])

In [213]:
vi = v0
for i in range(50):
    norm_vi =round(sqrt(vi.map(lambda line: pow(line[-1],2)).sum()),3)
    n_vi = vi.map(lambda v:(v[0], v[1]/norm_vi))
    vi=dot_prod(n_vi)
    m_v_ = file_M_bis.join(vi).sortByKey()
    m_v=m_v_.map(lambda line: (line[0][0], line[1])).map(lambda line: (line[0],line[1][0]*line[1][1]))
    m_v=m_v.reduceByKey(lambda a,b : round(a+b,3))
    vi=m_v
    l = vi.join(n_vi).map(lambda line:(line[0], line[1][0]*line[1][1])).map(lambda line: line[1]).sum()
    print(l)

6.573481738497154
4.863890721886076
5.2279363614867105
6.033690986347139
6.604231943076061
6.882367027729716
7.005300333950834
7.056887826460993
7.080761688863736
7.0919423810196305
7.095782853919911
7.098197576099211
7.099930684699915
7.099109577464789
7.1004371038174385
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
7.100288732394366
